In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci
from cosmatrix import cosm,expm_pade
expm = expm_pade
from autograd.extend import primitive, defvjp
from qutip import *
def annihilation(dim):
    return np.diag(np.sqrt(np.arange(1,dim)),1)
def creation(dim):
    return np.diag(np.sqrt(np.arange(1,dim)),-1)
def SNAIL_H(phi_ex,beta,N,Ej,Ec):
    phi_ex = 2*np.pi*phi_ex
    @primitive
    def Us_min(phi_ex):
        def U_s(phi): 
            return (-beta*np.cos(phi-phi_ex)-N*np.cos((phi)/N))
        phi_min = sci.optimize.minimize(U_s,0).x
        return phi_min
    
    def phi_minde(ans, phi_ex):
        def phi_minde_vjp(g):
            c2 = beta*np.cos(ans - phi_ex) + 1/N*np.cos(ans/N)
            return g*beta*np.cos(ans - phi_ex)/c2
        return phi_minde_vjp
    defvjp(Us_min, phi_minde)
    
    phi_min = Us_min(phi_ex)
    # potential expansion around minimum
    c2 = beta*np.cos(phi_min - phi_ex) + 1/N*np.cos(phi_min/N)
    omega_s = np.sqrt(8*c2*Ej*Ec)
    phi_zpf = np.power(2*Ec/(Ej*c2),1/4)
    g2 = Ej*phi_zpf**2*c2/2
    sdim = 20
    s = annihilation(sdim)
    sd = creation(sdim)
    x2 = np.matmul(s+sd,s+sd)
    c3 = (N**2-1)/N**2*np.sin(phi_min/N)
    g3 = Ej*phi_zpf**3*c3/3/2
    c4 = -beta*np.cos(phi_min-phi_ex) -1/N**3*np.cos(phi_min/N)

    g4 = Ej*phi_zpf**4*c4/4/3/2
    Hs = (omega_s * np.matmul(sd,s)
        - Ej*(beta*cosm(phi_zpf*(s+sd)+(phi_min-phi_ex)*np.identity(sdim))
        + N*cosm((phi_zpf*(s+sd)+phi_min*np.identity(sdim))/N))- g2*x2)
    energy,U = np.linalg.eigh(Hs)
    U_dag = np.conjugate(U.transpose())
    Hs = U_dag@Hs@U
    return Hs-energy[0]*np.identity(sdim),-1j*U_dag@(s-sd)@U/2/phi_zpf,phi_zpf,g3,g4
phi_ex = 0.423
def SNAIL_trunc(phi_ex,beta,N,Ej,Ec,app,order = 1):
    sdim=20
    phi_ex = 2*np.pi*phi_ex
    def U_s(phi): 
        return (-beta*np.cos(phi-phi_ex)-3*np.cos((phi)/3))
    phi_min = sci.optimize.minimize(U_s,0).x
    # potential expansion around minimum
    firstd = ad.grad(U_s)
    secondd = ad.grad(firstd)
    thirdd = ad.grad(secondd)
    fourd = ad.grad(thirdd)
    fived = ad.grad(fourd)
    sixd = ad.grad(fived)
    sevend = ad.grad(sixd)
    eightd = ad.grad(sevend)
    c2 = secondd(phi_min[0])
    c3 = thirdd(phi_min[0])/3/2
    c4 = fourd(phi_min[0])/4/3/2
    c5 = fived(phi_min[0])/5/4/3/2
    c6 = sixd(phi_min[0])/6/5/4/3/2
    c7 = sevend(phi_min[0])/7/6/5/4/3/2
    c8 = eightd(phi_min[0])/8/7/6/5/4/3/2
    omega_s = np.sqrt(8*c2*Ej*Ec)
    phi_zpf = np.power(2*Ec/Ej/c2,1/4)
    g2 = Ej*phi_zpf**2*c2/2
    g3 = Ej*phi_zpf**3*c3
    g4 = Ej*phi_zpf**4*c4
    g5 = Ej*phi_zpf**5*c5
    g6 = Ej*phi_zpf**6*c6
    g7 = Ej*phi_zpf**7*c7
    g8 = 1.5*Ej*phi_zpf**8*c8
    
    s = annihilation(sdim)
    sd = creation(sdim)
    Hs = omega_s * np.matmul(sd,s)
    x2 = np.matmul(s+sd,s+sd)
    x3 = np.matmul(s+sd,x2)
    x4 = np.matmul(s+sd,x3)
    x5 = np.matmul(s+sd,x4)
    x6 = np.matmul(s+sd,x5)
    x7 = np.matmul(s+sd,x6)
    x8 = np.matmul(s+sd,x7)
    Hs += g3*x3 + g4*x4 + g5*x5 + g6*x6 +1e-10*x8
    energy,U = np.linalg.eigh(Hs)
    U_dag = np.conjugate(U.transpose())
    Hs = U_dag@Hs@U
    if app == True:
        Hc = int_swt(g3,g4,g5,g6,omega_s,s,sd,order)
    return Hs-energy[0]*np.identity(sdim),Hc,phi_zpf
from qutip import qload
from sympy.physics.secondquant import *
from sympy import symbols
import sympy as sp
from cosmatrix import expm_pade
def int_swt(g3,g4,g5,g6,omegas,s,sd,order):    
    s1,s2 = qload('./symbolic/swt_charge')
    omegasp = symbols('omega_s', real=True)
    g3sp = symbols('g3', real=True)
    g4sp = symbols('g4', real=True)
    g5sp = symbols('g5', real=True)
    g6sp = symbols('g6', real=True)
    kappa = symbols('kappa', real=True)
    ind_a = symbols('s')
    a = B(ind_a)
    ad = Bd(ind_a)
    def replace_scalars_with_valued_matrices(expr, matrix_values):
        """
        Replace scalar symbols and scalar operations in a SymPy scalar expression with valued matrices.

        Args:
        expr (sympy.Expr): The input scalar expression.
        matrix_values (dict): A dictionary mapping scalar symbols to valued matrices.

        Returns:
        sympy.Expr: The modified expression with valued matrices.
        """

        def map_operations(node):
            if (isinstance(node, AnnihilateBoson) or isinstance(node, CreateBoson) )and node in matrix_values:
                return matrix_values[node]
            elif isinstance(node, sp.Add):
                return sp.Add(*[map_operations(arg) for arg in node.args])
            elif isinstance(node, sp.Mul):
                return sp.Mul(*[map_operations(arg) for arg in node.args])
            elif isinstance(node, sp.Pow):
                base, exponent = map_operations(node.base), map_operations(node.exp)
                return base**exponent
            else:
                return node

        return map_operations(expr)
    def smatrix(expr):
        expr = expr.subs({ g3sp:g3,g4sp:g4,g5sp:g5,g6sp:g6,kappa:1,omegasp:omegas})
        sp_s = sp.Matrix(s)
        sp_sd = sp.Matrix(sd)
        matrix_values = {a: sp_s, ad: sp_sd}
        result = replace_scalars_with_valued_matrices(expr, matrix_values)
        return np.array(result,np.complex128)
    if order == 1:
        app = smatrix(s1)
    if order == 2:
        app = smatrix(s1) + smatrix(s2) 
#     if order == 3:
#         D = smatrix(s1) + smatrix(s2) + smatrix(s3)
    return app

In [2]:
def find_optimal_k(A, B, D):
    # Define a large initial minimum difference
    min_diff = float('inf')
    optimal_k = None
    
    # Iterate over a range of possible k values
    # The range can be adjusted based on expected size of k or other insights you have about your problem
    for k in range(-1000, 1000):
        # Calculate the difference for this value of k
        diff = abs(A - (B + k * D))
        
        # If this is the smallest difference we've found so far, update min_diff and optimal_k
        if diff < min_diff:
            min_diff = diff
            optimal_k = k
            
    return optimal_k
# Function to calculate overlap (you might use inner product, fidelity, etc.)
def calculate_overlap(state1, state2):
    return abs((state1.dag() * state2)[0,0])**2

In [3]:
import qutip as qt
def floquet_spec(omega,A,trunc = False, app = False, order=1):
    beta = 0.12
    N = 3
    Ej = 70*2*np.pi
    Ec = 0.177*2*np.pi
    
    dims = sdim =  20
    s = annihilation(dims)
    sd = creation(dims)
    if trunc == False:
        H0,Hc,phi_zpf,g3,g4 = SNAIL_H(phi_ex,beta,N,Ej,Ec)
    else:
        H0,Hc,phi_zpf= SNAIL_trunc(phi_ex,beta,N,Ej,Ec,app,order)
        Hc = (-1j*(s-sd))/2/phi_zpf

    energies,U=np.linalg.eig(H0)
    energies = energies - energies[0]*np.ones(len(energies))
    energies = energies/2/np.pi
    # U_dag = np.conjugate(U.transpose())
    # U_dag@Hc@U
    energies = np.sort(energies)
    omega_s = energies[1]
    H0 = Qobj(H0)
    Hc = Qobj(Hc)
    args = {'w': omega}
    T = (2*np.pi)/omega
    H = [H0, [Hc, lambda t, args: A*np.cos(args['w']*t)]] 
    options = Options(nsteps=1000000)  # Increasing nsteps to 10000, adjust as needed
    f_modes, f_energies = floquet_modes(H, T, args, False, options=options) 
    
    # Define your system dimension
    
    floquet_states = f_modes
    # Create your basis states
    basis_states = [qt.basis(sdim, i) for i in range(3)]
    # Initialize a list to hold the indices of the Floquet states with the maximum overlap for each basis state
    max_overlap_indices = [-1] * 3
    max_overlaps = [0] * 3
    # Loop over each Floquet state
    for f_index, f_state in enumerate(floquet_states):

        # Loop over each basis state
        for b_index, b_state in enumerate(basis_states):

            # Calculate the overlap
            overlap = calculate_overlap(f_state, b_state)

            # If this is the maximum overlap for this basis state, store the index
            if overlap > max_overlaps[b_index]:
                max_overlaps[b_index] = overlap
                max_overlap_indices[b_index] = f_index
    print(max_overlaps)
    energy01 = f_energies[max_overlap_indices[1]]/2/np.pi-f_energies[max_overlap_indices[0]]/2/np.pi
    k = find_optimal_k(omega_s, energy01, omega/2/np.pi)
    energy01 =energy01+k*omega/2/np.pi
    omega_ss = energy01-omega_s
    energy02 = f_energies[max_overlap_indices[2]]/2/np.pi-f_energies[max_overlap_indices[0]]/2/np.pi
    k = find_optimal_k(2*omega_s, energy02, omega/2/np.pi)
    energy02 =energy02+k*omega/2/np.pi
    anh = energy02 - energy01 - energy01
    if (max_overlaps<(np.sqrt(2)/4*np.ones(3))).all():
        omega_ss = 1000
        anh = 1000
    return np.abs(omega_ss),anh

In [4]:

floquet_spec(4*2*np.pi,0.5*2*np.pi,False)

[0.8548831877781256, 0.6146666864803467, 0.43906109846923935]


(0.0038285398105228907, 0.018336416546873124)